# Intro
* familiar bit: few shot, chain of reasoning style prompt
* works with off the shelf models, no fine tuning, you can use this method with LLMs you get over api calls.
* TODO: screenshot of cot prompt

# Algorithm
* screen shot of algo demo from paper
* picture of Jon's diagram
       

# code
babbage results

In [4]:
%%capture
! pip install openai

In [5]:
from openai import Completion
import openai

In [6]:
openai.api_key_path = '/root/.openai/key'

In [102]:
# note: this works with greedy decode
few_shot_prompt='''Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, theyhad 74 - 35 = 39. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The answer is 8.
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is 9.
Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
A: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is 29.
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?
A: Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 28 - 15 dollars left. 28 - 15 = 8. The answer is 8.
'''

In [85]:
solvable_q='''Q: Alex had 18 cars, then doubled how many cars he had, then lost one-sixth of the cars, and finally gained one more. How many cars does he have at the end?
A: '''

In [86]:
sometimes_solvable_q='''Q: Kyle bought last year's best-selling book for $20. This is after a 25% discount from the original price. What was the original price of the book?
A: '''

In [112]:
sometimes_solvable_q='''Q: Kyle bought last year's best-selling book for $21. Next year it will cost 35% more. How much will it cost next year?
A: '''

single = Completion.create(prompt=(few_shot_prompt + sometimes_solvable_q),
                      model="davinci-002", max_tokens=75, n=1, temperature=0.0, stop="Q: ")
single['choices'][0]['text']

'35% more means 35% + 100% = 135%. So the price will be 21 x 1.35 = 28.05. The answer is 28.05.\n'

In [124]:
completions = Completion.create(prompt=(few_shot_prompt + sometimes_solvable_q),
                      model="davinci-002", top_p=.95, max_tokens=75, n=40, stop="Q: ")

In [125]:
for response in completions['choices'][:3]:
    pass
    #print('=====\n' + response['text'])



In [126]:
import re
pattern = re.compile(r'answer is ([^\s]+)')
answers = []
for response in completions['choices']:
    matches = pattern.findall(response['text'])
    if len(matches) > 0:
        answers.append(matches[0])

from collections import Counter
ctr = Counter(answers)
ctr.most_common(5)

[('28.35.', 7), ('28.05.', 6), ('$28.05.', 3), ('28.35', 2), ('$28.05', 1)]

# Results
## parameters
40 samples, temperature sampling t=.5, topk=40, but results compatible with many sampling strategies
## specific result improvement example
davinci-002 / GSM8k, 60.1 -> 78, + 17.9% difference
screenshot of the gsm8k question

## Screenshot of the greedy vs. sampled paths for palm-540b
screenshot of row example from appendix, palm also saw 17.9% improvement

## table of results
lots of info

# Compare to other approaches 
* multimodel ensemble - only 3, unsure exactly how algo worked for this, but did worse
* ensemble of prompts - confusing
* beam search (used by itself)

# Effects of parameters
* num reasoning paths
  * perf saturation after 10
* temperature, top k, top p
* sampling method - beam search works, but has less diversity than other methods

# Observations
* bigger effect with higher model size
* consistency % is confidence indicator
  * show graph
* trade 1 time compute at train for permanent 10x or more compute cost increase